# 銀イルカの3人のセリフをスクレイピングして，同じところと違うところを

とりま wikiとかから智絵里のセリフをスクレイピングする． 

テキスト解析 -> 出現頻度，


セリフをword2vecで学習して，分散表現を計算できるようにする

分散表現を次元削減して2次元座標上にプロットすることで，単語ごとの類似度を可視化する

銀イルカ3人のセリフをスクレイピング  

doc2vecで学習して，

In [1]:
import os
import re
import time
import random
import requests
from bs4 import BeautifulSoup
import pandas as pd

/home/wani/dere/idol_info/.venv/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/wani/dere/idol_info/.venv/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
pd.options.display.max_rows = 500
pd.options.display.max_columns = 20 

## デレステセリフのスクレイピング

In [3]:
def get_quotes_sute(q):
    """
    デレステで公開されている各アイドルのセリフデータを取得する
    [arg]
    q: int
    [ret]
    df: DataFrame
    """
    html = f"https://starlight.kirara.ca/char/{q}"
    r = requests.get(html)
    r.encoding = r.apparent_encoding
    soup = BeautifulSoup(r.text, "lxml")
    texts = soup.find_all("span")
    corpus = []
    for text in texts:
        corpus.append(text.text)
    df = pd.DataFrame(corpus)
    #df[0].value_counts().head(10) # 値の出現頻度をカウント
    df = df[~df.duplicated()] # 重複行をdataframeから除外
    df = df[~df[0].str.match("^[\s|a-zA-Z|<|\+|\d$]")] # 正規表現にマッチするものを除外
    df = df[df[0] != ""]
    return df.reset_index(drop=True)

## モバマスセリフのスクレイピング

In [4]:
def get_quotes_moba(query="緒方智絵里", debug=False):
    """
    モバマスで公開されているセリフ（含 アイドル名、カード名 など）
    をスクレイピングして pandas dataframe にする
    
    [args]
    query: str 省略すると 緒方智絵里 のセリフをスクレイピングする
    debug: Bool Trueにするとスクレイピング数を上限2にする。それだけ もっと良い使い方あるはず...
    [ret]
    df: DataFrame
    """
    
    html = f"https://icws.indigo-bell.com/search?q={query}&st=n"
    r = requests.get(html)
    r.encoding = r.apparent_encoding
    soup = BeautifulSoup(r.text, "lxml")
    a = soup.find("a", string="Last　»").get("href")
    if debug == True:
        n = 2
    else:
        n = int(re.search(r"page=(\d+)&", a).group(1)) # クロールするページ数
    
    df = pd.DataFrame()
    for i in range(n): # セリフが登録されてるページ分ループする
        print(f"crawling page: {i + 1}/{n}")
        html = f"https://icws.indigo-bell.com/search?page={i + 1}&q={query}&st=n"
        r = requests.get(html)
        r.encoding = r.apparent_encoding
        soup = BeautifulSoup(r.text, "lxml")
        table = soup.find_all("tbody",class_="result")[0]
        rows = table.find_all("tr")
        tmp = []
        tm = 5 + random.randint(0, 5)

        for row in rows:
            cell_txt = []
            for cell in row.find_all(["td", "th"]):
                cell_txt.append(cell.text)
            tmp.append(cell_txt)
        df = df.append(tmp)
        #print(f"sleep: {tm}")
        if i != (n - 1):
            time.sleep(tm)
        else:
            continue
    return df.reset_index(drop=True) # indexを振り直してもとのindexも削除

In [ ]:
# deresute id
# chieri : 112
# minami : 185
# yui    : 246
# sae    : 132
# hotaru : 161
# yoshino: 262

In [ ]:
path = "data/"
aa = get_quotes_sute(112)
#bb = get_quotes_moba(query="緒方智絵里", debug=True) # test
bb = get_quotes_moba(query="緒方智絵里")

## モバ，ステの結合と正規化

In [10]:
bb[3] = bb[3].str.replace("○○", "") # OO (プロデューサー名)部分を落とす
bb[3] = bb[3].str.replace("〇〇", "") # OO (プロデューサー名)部分を落とす
# aa + bb のコーパスを作る
docs = pd.Series()
docs = pd.concat([aa[0], bb[3]])
docs = docs.str.replace("プロデューサー", "PRODUCER")
docs = docs.str.replace("[Producer]", "PRODUCER", regex=False)
docs = docs.reset_index(drop=True)

In [11]:
docs.to_csv('data/corpus.csv')

/home/wani/dere/idol_info/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


## 形態素解析

mecabを使う。インストールしていなければめかぶをインストール

neeologdまでインストールする手順はの参考は
- [ubuntu 18.10 に mecab をインストール](https://qiita.com/ekzemplaro/items/c98c7f6698f130b55d53)
- [WSLでMeCabとJupyter Notebookを動かす](https://qiita.com/hironobukawaguchi3/items/61a8664dd26c1da66880)
- [MecabをPythonで使うまで](https://qiita.com/Sak1361/items/47e9ec464ccc770cd65c)

---

$ mecab -d /usr/lib/mecab/dic/mecab-ipadic-neologd

In [12]:
def extractNoun(text,
                mecab_mode = r'-Owakati -d /usr/lib/mecab/dic/mecab-ipadic-neologd',
                noun="名詞"):
    """
    [arg]: sentence
    [ret]: a list of noun words
    """
    tagger = MeCab.Tagger(mecab_mode)
    node = tagger.parseToNode(text)
    keywords = []
    while node:
        surface = node.surface # 表層形を表示(str)
        meta = node.feature.split(",") # ['記号', '読点', '*', '*', '*', '*', '、', '、', '、']
        if meta[0] == noun:
            keywords.append(node.surface)
        node = node.next # 次のノードのアドレスを指定
    return keywords

def tokenize(text, wakati=True, dic="neologd"):
    """
    テキストを形態素解析して分かち書きする
    [arg]
    text: str
    [ret]
    term: str
    """
    import MeCab
    if dic == "neologd":
        mecab_mode = MeCab.Tagger(r'-Owakati -d /usr/lib/mecab/dic/mecab-ipadic-neologd') # debugging
    else:
        mecab_mode = MeCab.Tagger(r'-Owakati')
    if wakati == True: # 分かち書きしたものをそのまま取り出す
        term = mecab_mode.parse(text).strip()
    else: # 品詞で
        term = extractNoun(text, mecab_mode)
    return term

In [13]:
import MeCab
text = "テキストを形態素解析して分かち書きする"
print(extractNoun(text))

['テキスト', '形態素解析', '分かち書き']


In [14]:
print(f'ipa-dic    ： {tokenize("特急はくたか", dic = "ipa")}')
print(f'neologd-dic： {tokenize("特急はくたか")}')

ipa-dic    ： 特急 は く た か
neologd-dic： 特急 はくたか


In [15]:
from gensim import corpora, models

In [16]:
docs_list = [tokenize(row).split() for row in docs]
dic = corpora.Dictionary(docs_list)
dic.save_as_text('./data/text.dict')  # 保存
#gensim.corpora.Dictionary.load_from_text('./data/text.dict')  # 次回からファイルロード可

In [17]:
print(len(dic)) # dictionaryに登録された単語(形態素)数

2949


In [18]:
# 単語出現頻度を数える
# 余談: bashだと $ sort -t$'\t' -k3 -nr text.dict で単語とその出現頻度回数カウントできる
from collections import defaultdict
freq = defaultdict(int)

for text in docs_list:
    for token in text:
        freq[token] += 1
print(sorted(freq.items(), key=lambda x:x[1], reverse=True))

[('…', 2974), ('、', 2241), ('て', 1102), ('の', 820), ('です', 810), ('。', 776), ('に', 758), ('さん', 634), ('た', 516), ('が', 507), ('で', 468), ('も', 453), ('は', 445), ('ます', 405), ('な', 380), ('し', 372), ('ん', 365), ('を', 357), ('と', 337), ('か', 330), ('ね', 317), ('!', 293), ('私', 277), ('PRODUCER', 268), ('から', 267), ('へ', 213), ('?', 206), ('ない', 175), ('って', 174), ('あの', 168), ('…。', 163), ('まし', 158), ('よ', 153), ('い', 148), ('♪', 146), ('お', 133), ('だ', 132), ('すっ', 128), ('「', 128), ('でも', 127), ('わたし', 127), ('たい', 125), ('え', 125), ('みんな', 122), ('？', 113), ('いい', 109), ('くれ', 109), ('けど', 108), ('たら', 104), ('幸せ', 103), ('なっ', 102), ('こと', 96), ('あ', 90), ('お仕事', 89), ('見', 86), ('あっ', 84), ('てる', 81), ('ませ', 80), ('アイドル', 76), ('っ', 75), ('みたい', 73), ('ください', 72), ('そう', 69), ('かも', 67), ('き', 66), ('う', 65), ('一緒', 65), ('この', 64), ('よう', 61), ('ま', 59), ('その', 58), ('だけ', 57), ('なぁ', 55), ('智絵里', 54), ('ちょっと', 53), ('」', 51), ('なら', 50), ('なんて', 48), ('とっても', 48), ('クローバー', 47),

In [1]:
import matplotlib.pyplot as plt

In [19]:
corpus = [dic.doc2bow(doc) for doc in docs_list] # コーパスの作成
corpora.MmCorpus.serialize('./data/text.mm', corpus)  # 保存

In [20]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

# 折角計算したのでpickleに保存
import pickle
with open('./data/corpus_tfidf.dump', mode='wb') as f:
    pickle.dump(corpus_tfidf, f)

# 次回からはロードできます
# with open('./data/corpus_tfidf.dump', mode='rb') as f:
#     corpus_tfidf = pickle.load(f)

In [21]:
tfidf.idfs

{0: 2.789845153769155,
 1: 10.603626344986193,
 2: 10.603626344986193,
 3: 10.603626344986193,
 4: 0.33099656000982236,
 5: 0.5055942620256653,
 6: 1.7644225568892482,
 7: 10.603626344986193,
 8: 3.8222666314615323,
 9: 7.603626344986193,
 10: 2.542930413298638,
 11: 5.43370134354388,
 12: 1.3652216056611133,
 13: 2.119810567721936,
 14: 10.603626344986193,
 15: 10.603626344986193,
 16: 3.9742697249065824,
 17: 10.603626344986193,
 18: 4.064467533878161,
 19: 2.4589681021543095,
 20: 3.7209832956243507,
 21: 4.9031866268451,
 22: 5.08006438892918,
 23: 5.111773248656517,
 24: 7.43370134354388,
 25: 1.446279409623349,
 26: 5.318224126123944,
 27: 2.4640749925873986,
 28: 5.696735749377674,
 29: 10.603626344986193,
 30: 10.603626344986193,
 31: 7.018663844265037,
 32: 7.603626344986193,
 33: 5.9031866268451,
 34: 8.603626344986191,
 35: 5.696735749377674,
 36: 9.603626344986191,
 37: 6.28169825009883,
 38: 8.28169825009883,
 39: 2.7148830960879327,
 40: 1.471769384377399,
 41: 3.82226663

In [22]:
# 文書量にもよりますが、数時間程度かかったりもします。
# '18/12/03追記: LdaMulticore で worker の数を増やせばかなり早くなるかもです
lda = models.LdaModel(corpus=corpus_tfidf, id2word=dic,
                             num_topics=50, minimum_probability=0.001,
                             passes=20, update_every=0, chunksize=10000)
lda.save('./data/lda.model')  # 保存
#lda = gensim.models.LdaModel.load('./data/lda.model')  # 次回からロード可

In [23]:
lda.print_topic(10)

'0.033*"ぇいっ" + 0.023*"景色" + 0.021*"い" + 0.017*"ここ" + 0.015*"いま" + 0.015*"しよう" + 0.015*"どう" + 0.013*"ど" + 0.012*"食べ" + 0.010*"飲み"'

In [24]:
for i in range(10):
    print('tpc_{0}: {1}'.format(i, lda.print_topic(i)))

tpc_0: 0.018*"何" + 0.016*"ください" + 0.016*"えっ" + 0.014*"見捨て" + 0.013*"も" + 0.012*"ツッコミ" + 0.012*"て" + 0.011*"ちゃんと" + 0.011*"あの" + 0.011*"いつか"
tpc_1: 0.018*"出来" + 0.017*"上手" + 0.016*"ぱたぱた" + 0.016*"扇い" + 0.015*"ラッキー" + 0.013*"あげ" + 0.011*"ふぁ" + 0.011*"もっともっと" + 0.010*"ち" + 0.010*"双子座"
tpc_2: 0.030*"よう" + 0.023*"とき" + 0.023*"ドレス" + 0.023*"楽しん" + 0.018*"夏" + 0.016*"似合う" + 0.015*"出" + 0.015*"受け止め" + 0.014*"事" + 0.014*"歌い"
tpc_3: 0.043*"ドキドキ" + 0.020*"うん" + 0.016*"時" + 0.016*"ら" + 0.012*"学園" + 0.011*"シロツメクサ" + 0.009*"積もっ" + 0.006*"冷め" + 0.006*"ぬるい" + 0.005*"吹い"
tpc_4: 0.033*"初めて" + 0.024*"もう" + 0.022*"浴衣" + 0.018*"ちょっと" + 0.018*"こんなに" + 0.016*"お仕事" + 0.012*"か" + 0.012*"?" + 0.012*"なんだか" + 0.012*"だっ"
tpc_5: 0.020*"成長" + 0.018*"わ" + 0.018*"っぽい" + 0.017*"心配" + 0.014*"ありません" + 0.011*"寂しく" + 0.010*"明日" + 0.008*"も" + 0.008*"あの" + 0.008*"コート"
tpc_6: 0.027*"あり" + 0.019*"ば" + 0.016*"外" + 0.015*"伝わり" + 0.012*"アクセサリー" + 0.012*"方" + 0.012*"忘れ" + 0.012*"どうぞ" + 0.011*"やってこ" + 0.010*"出会え"
tpc_7: 0.021*"た" +